In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn import metrics
from sklearn.metrics.cluster import contingency_matrix
from sklearn.model_selection import train_test_split

In [ ]:
from DBSCAN import DBSCAN as c_DBSCAN
from Agglomerative import Agglomerative as c_Agglomerative
c_KMeans = __import__('Draft K-Means').KMeans

In [ ]:
df = pd.read_csv('dataset/iris.data', header=None)

In [ ]:
df.head()

In [ ]:
x = df.drop([4], axis=1)
y = df[4]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
def get_cluster_mapping_to_target(x, y, label):
    print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, label))
    print("Completeness: %0.3f" % metrics.completeness_score(y, label))
    print("V-measure: %0.3f" % metrics.v_measure_score(y, label))

## DBSCAN

In [ ]:
cdbscan_model = c_DBSCAN(x_train, 0.8, 2)
cdbscan_label = cdbscan_model.get_all()

In [ ]:
# cdbscan_map = get_cluster_mapping_to_target(x_train, y_train, cdbscan_label)

In [ ]:
model = DBSCAN(eps=0.8, min_samples=2).fit(x_train)
model.labels_

## Agglomerative

### Single

In [ ]:
cagglomerative_single_model = c_Agglomerative(x_train, 3, 'single')
cagglomerative_single_label = cagglomerative_single_model.get_all()

In [ ]:
cagglomerative_single_map = get_cluster_mapping_to_target(x_train, y_train, cagglomerative_single_label)

### Complete

In [ ]:
cagglomerative_complete_model = c_Agglomerative(x_train, 3, 'complete')
cagglomerative_complete_label = cagglomerative_complete_model.get_all()

In [ ]:
cagglomerative_complete_map = get_cluster_mapping_to_target(x_train, y_train, cagglomerative_complete_label)

### Average

In [ ]:
cagglomerative_average_model = c_Agglomerative(x_train, 3, 'average')
cagglomerative_average_label = cagglomerative_average_model.get_all()

In [ ]:
cagglomerative_average_map = get_cluster_mapping_to_target(x_train, y_train, cagglomerative_average_label)

### Average Group

In [ ]:
cagglomerative_avg_g_model = c_Agglomerative(x_train, 3, 'average_group')
cagglomerative_avg_g_label = cagglomerative_avg_g_model.get_all()

In [ ]:
cagglomerative_avg_g_map = get_cluster_mapping_to_target(x_train, y_train, cagglomerative_avg_g_label)

## KMeans

In [ ]:
ckmeans_model = c_KMeans(3, 1e-8, 1000)
ckmeans_label = ckmeans_model.fit_predict(x_train)

In [ ]:
print(ckmeans_label)

In [ ]:
kmeans_model = KMeans(3, tol=1e-8)
kmeans_model.fit(x_train)

In [ ]:
kmeans_model.labels_